In [55]:

import os
from dotenv import load_dotenv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [56]:
import os
from dotenv import load_dotenv
from groq import ChatGroq

# Print the current working directory
print(f"Current working directory: {os.getcwd()}")

# Specify the path to the .env file
env_path = '.env'

# Load environment variables from the .env file
load_dotenv(env_path)

# Retrieve the API key from the environment variables
api_key = os.getenv('GROQ_API_KEY')

# Debug: Print the API key to ensure it's being loaded correctly
print(f"GROQ_API_KEY: {api_key}")

# Check if the API key is None and raise an error if it is
if api_key is None:
    raise ValueError("GROQ_API_KEY environment variable is not set. Please check your .env file and ensure it contains the correct key-value pair.")

# Initialize the ChatGroq client with the API key and other parameters
chat = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama3-70b-8192")

# Example usage of the ChatGroq client
response = chat.chat_completion(messages=[{"role": "user", "content": "Explain the importance of fast language models"}])
print(response.choices[0].message.content)





Current working directory: /Users/kiri/Documents/HWR/master/SS/Text & Web/Rag_project-1/first attempt rag
GROQ_API_KEY: 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


AttributeError: 'ChatGroq' object has no attribute 'completions'

In [ ]:
#load_dotenv()

False

In [ ]:
#api_key = os.getenv('GROQ_API_KEY')

In [ ]:
#chat = ChatGroq(temperature=0, groq_api_key=api_key, model_name="llama3-70b-8192")

ValidationError: 1 validation error for ChatGroq
__root__
  Did not find groq_api_key, please add an environment variable `GROQ_API_KEY` which contains it, or pass `groq_api_key` as a named parameter. (type=value_error)

In [ ]:
persist_directory = 'chroma_db'
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/paraphrase-MiniLM-L6-v2')

/opt/homebrew/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/opt/homebrew/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [ ]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding_model,
                  collection_name="title_abstract_chroma_db")

NameError: name 'persist_directory' is not defined

In [ ]:
retriever = vectordb.as_retriever()

In [ ]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

prompt = set_custom_prompt()

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=chat,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt}
)

NameError: name 'chat' is not defined

In [ ]:
query = "Give me 2 articles that talk about ad blockers."
result = qa({"query": query})

NameError: name 'qa' is not defined

In [ ]:
print("Answer:", result["result"])

Answer: Here are 2 articles that talk about ad blockers:

1. Ad-Blockers: A Blessing or a Curse?
2. (No other article talks about ad blockers)


In [ ]:
query = "Give me articles that talk about software innovation."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Essence: facilitating software innovation" talks about software innovation.


In [ ]:
query = "What is article with the name Data Science for Scoial Good about."
result = qa({"query": query})
print("Answer:", result["result"])

Answer: The article "Data Science for Social Good" is about the diminishing emphasis on social good challenges in data science research, and presents a framework for "data science for social good" research that considers the interplay between relevant data science research genres, social good challenges, and different levels of sociotechnical abstraction.


In [ ]:
import textwrap


In [ ]:
query = "What is the article with the name Data Science for Social Good about."
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 The article "Data Science for Social Good" is about presenting a framework for
"data science for social good" (DSSG) research that considers the interplay
between relevant data science research genres, social good challenges, and
different levels of sociotechnical abstraction, and highlighting the lack of
research focus on social good challenges in the field of data science.


In [ ]:
query = "Give me full abstract of article Data Science for Social Good."
result = qa({"query": query})
wrapped_answer = textwrap.fill(result["result"], width=80)
print("Answer:\n", wrapped_answer)

Answer:
 Here is the abstract of the article "Data Science for Social Good":  Data
science has been described as the fourth paradigm of scientific discovery. The
latest wave of data science research, pertaining to machine learning and
artificial intelligence (AI), is growing exponentially and garnering millions of
annual citations. However, this growth has been accompanied by a diminishing
emphasis on social good challenges-our analysis reveals that the proportion of
data science research focusing on social good is less than it has ever been. At
the same time, the proliferation of machine learning and generative AI has
sparked debates about the sociotechnical prospects and challenges associated
with data science for human flourishing, organizations, and society. Against
this backdrop, we present a framework for "data science for social good" (DSSG)
research that considers the interplay between relevant data science research
genres, social good challenges, and different levels of sociot